# Pruebas y explicaciones de load.py y duckDB

In [1]:
!pip install duckdb
from dw import DW
import duckdb


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\david\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


El código siguiente importa la clase `DW` de `dw.py`, que es la encargada de crear la base de datos de duckdb con todas las tablas y relaciones de nuestro modelo. Si el argumento `create = True`, entonces si ya existía la base de datos la borrará y creará otra, es como reiniciarlo todo. 

Una vez hecha la clase y la base de datos, la variable `conn`, que guardamos en `conn`, es la conexión mediante la cual haremos las queries. En este caso no existe el objeto cursor y la conexión hace su función. 

In [2]:
dw = DW(create = True)
conn = dw.conn_duckdb

Connection to the DW created successfully
Tables created successfully


Ahora podemos empezar a hacer queries, como estas por ejemplo: 

In [3]:
# Mostrar tablas
conn.execute("SHOW TABLES").fetchall()

[('aircrafts',),
 ('daily_kpis',),
 ('days',),
 ('monthly_kpis',),
 ('months',),
 ('reporters',)]

In [4]:
# Consultar todos los elementos que hay en una tabla
conn.execute(  
    """SELECT * FROM aircrafts"""
).fetchall()


[]

In [5]:
# Añadir filas a una tabla
conn.execute(  
    """
    INSERT OR IGNORE INTO aircrafts VALUES 
            ('avion_de_david', 'dinosaudio', 'david'),
            ('avioneta', 'rayaner', 'fabrica'),
            ('dron_volador', 'tesla', 'elonmusk')
    """
)

In [6]:
#Eliminar todas las filas
conn.execute( """DELETE FROM aircrafts""" )
conn.commit()

Por lo que he visto, este último bloque no funciona bien con el segundo. Cuando añades los elementos, los eliminas y vuelves a intentar ponerlos, da un error de integridad de la clave primaria. 

Parece ser que como en la definición de la tabla se dice que es una `CachedDimension`, pygrametl se guarda las claves primarias recientes en una cache, pero al eliminar los elementos no las elimina sus claves de aquí y piensa que siguen existiendo. Si durante el proyecto resuta muy importante eliminar elementos habrá que crear algun método dentro de `DW` para gestionar esta cache o buscar otra forma de hacer las cosas. Pero como lo único que hay que hacer es loadear lo que pasa el transform, es probable que no. 

## Obtener KPIs

En teoría llamando a estas funciones deberíamos obtener los KPIs desde la base de datos de duckdb. Debería ser más eficiente que con el `extract.py`. Sin embargo ahora no funciona ninguna. 

In [8]:
dw.query_utilization()

ParserException: Parser Error: syntax error at or near "="

LINE 4:             WHERE dk.registration = mk.registration = a.registration AND dk.day = d.day AND mk.month = m.month...
                                                            ^

In [9]:
dw.query_reporting()

BinderException: Binder Error: column "rrh" must appear in the GROUP BY clause or must be part of an aggregate function.
Either add it to the GROUP BY list, or use "ANY_VALUE(rrh)" if the exact value of "rrh" is not important.

LINE 2:             SELECT a.manufacturer, m.year, mk.rrh, mk.rrc
                                                   ^

In [11]:
dw.query_reporting_per_role()

BinderException: Binder Error: column "rrh" must appear in the GROUP BY clause or must be part of an aggregate function.
Either add it to the GROUP BY list, or use "ANY_VALUE(rrh)" if the exact value of "rrh" is not important.

LINE 2:             SELECT a.manufacturer, m.year, r.role, mk.rrh, mk.rrc
                                                           ^

In [12]:
conn.close()